In [1]:
import os
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
import requests

GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")
if not GITHUB_TOKEN:
    raise ValueError("Set GITHUB_TOKEN environment variable to a GitHub PAT")
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_0,
    rate_limit=5_00,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

    # Depending on your internet connection, this can take several minutes to run...
fetch_issues()


  0%|          | 0/1 [00:00<?, ?it/s]

Downloaded all the issues for datasets! Dataset stored at ./datasets-issues.jsonl


In [10]:
from datasets import load_dataset
issues_dataset = load_dataset("json", data_files="datasets-issues.jsonl", split="train")
split_data = issues_dataset.train_test_split(test_size=0.2) 
split_data

In [3]:
sample = issues_dataset.shuffle(seed=666).select(range(3))

# Print out the URL and pull request entries
for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/pull/6871
>> Pull request: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/6871', 'html_url': 'https://github.com/huggingface/datasets/pull/6871', 'diff_url': 'https://github.com/huggingface/datasets/pull/6871.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/6871.patch', 'merged_at': datetime.datetime(2024, 5, 6, 9, 25, 52)}

>> URL: https://github.com/huggingface/datasets/pull/6925
>> Pull request: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/6925', 'html_url': 'https://github.com/huggingface/datasets/pull/6925', 'diff_url': 'https://github.com/huggingface/datasets/pull/6925.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/6925.patch', 'merged_at': None}

>> URL: https://github.com/huggingface/datasets/issues/6841
>> Pull request: None



In [4]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)
issues_dataset

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'draft', 'pull_request', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'is_pull_request'],
    num_rows: 100
})

In [5]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    return [r["body"] for r in response.json()]


# Test our function works as expected
get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

In [6]:
# Depending on your internet connection, this can take a few minutes...
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
issues_with_comments_dataset.push_to_hub("github-issues")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/skyloevil/github-issues/commit/116ebfd7aca660c7c58a5068362e225822901a43', commit_message='Upload dataset', commit_description='', oid='116ebfd7aca660c7c58a5068362e225822901a43', pr_url=None, pr_revision=None, pr_num=None)